## **Installing Packages**

In [1]:
!pip install textblob
!pip install sentencepiece  
!pip install transformers
!pip install textstat
!pip install language-tool-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Restart Runtime after installing 

## **Importing Packages**

In [6]:
import re
import nltk 
import spacy
import textstat
import numpy as np
import pandas as pd
import seaborn as sn 
from textblob import Word
import matplotlib.pyplot as plt
import language_tool_python

from nltk.tag import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nlp = spacy.load("en_core_web_sm")
tool = language_tool_python.LanguageTool('en-US')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Reading Data From Google Drive**

In [7]:
from google.colab import drive
drive.mount('/content/drive')
Data_Essay_01 = pd.read_excel("/content/drive/MyDrive/IntelliTech-DataSet/valid_set.xlsx")
# This line is intended for valid_set.xlsx only
Data_Essay_01 = Data_Essay_01.query('essay_set == 1')
print(Data_Essay_01.shape)
Data_Essay_01.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(589, 5)


,essay_id,essay_set,Essay,domain1_predictionid,predicted_score
0,1788,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788,7
1,1789,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789,8
2,1790,1,"Dear Local newspaper, Have you been spending a...",1790,9
3,1791,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791,9
4,1792,1,"Dear newspaper, I strongly believe that comput...",1792,9


# **Feature Extraction**

## **Essay Pre Processing**

In [8]:
def Remove_NER(Essay):
  """
    Removes Named Entity Recognition (NER) from each essay

    Args:
      Sentence: Essay of each student 
    
    Returns: 
      String

  """
  token = Essay.split()
  newTokens = []
  # print(token)
  for i in range(len(token)):
    if "@" in token[i]:
      if "," in token[i]:
        newTokens.append(",")
      elif "." in token[i]:
        newTokens.append(".")
      elif "!" in token[i]:
        newTokens.append("!")
    else:
      newTokens.append(token[i])

  return " ".join(newTokens)
  # return ' '.join (word for word in Essay.split(' ') if not word.startswith('@'))
def Remove_NER2(Essay):
    return ' '.join (word for word in Essay.split(' ') if not word.startswith('@'))
def Remove_Punctuations(sentence):
  """
    Removes punctuations from text
    Args:
      sentence: Essay of each student
    
    Returns: 
      String
  """
  punctuations = '''!()-[]{};:"\,/'<>.?@#$%^&*_~'''
  newSentence = ""
  for word in sentence:
      if (word in punctuations):
          newSentence = newSentence + " "
      else: 
          newSentence = newSentence + word
  return newSentence

def LowerCase_Words(Essay):
  """
    Lower case all the words in an essay

    Args:
      Sentence: Essay of each student
    
    Returns: 
      String
  """
  return re.sub('[0-9]+','', Essay).lower() 

def Tokenize_Essay(Essay):
    """
      Create Tokens of each Essay

      Args:
        Essay: Essay of each student
      
      Returns: 
        String
    """
    Preprocessed = Remove_Punctuations(Essay)
    return " ".join(word_tokenize(Preprocessed))

def Remove_White_Spaces(Essay):
  """
    Removes Extra White Spaces

    Args:
      Essay: Essay of each student
    
    Returns: 
      String
  """
  return " ".join(Essay.split())

def Remove_Special_Characters(Essay):
  """
    Removes Special Characters from Essay

    Args:
      Essay: Essay of each student
    
    Returns: 
      String
  """
  new_text = re.sub(r"[^a-zA-Z0-9 ]", "", Essay)
  return new_text

## **Basic Count Features**

This section will cover:


*   Counting Sentences per Essay
*   Counting Words per Essay
*   Counting Characters per Essay
*   Average Words per Essay
*   Counting Syllables
*   Percentage Of Hard Words




#### Counting Sentences per Essay

In [9]:
def Sentence_Count(Essay):
    """
    Counts sentences in an essay

    Args:
      Essay: Essay of each student 
    
    Returns: 
      int
  """
    sentence_no = nltk.sent_tokenize(Essay)
    return len(sentence_no)

In [10]:
Data_Essay_01['Sent_Count'] = Data_Essay_01['Essay'].apply(Sentence_Count)
Data_Essay_01.sample()

,essay_id,essay_set,Essay,domain1_predictionid,predicted_score,Sent_Count
514,2306,1,Crunch! That is the sound of another kid eatin...,2306,9,30


#### Counting Words per Essay

**Observation:** These word count are more than the original count coz of nltk tokenization. Punctations are treated as seperate words.


In [11]:
def Word_Count(Essay):
  """
    Counts words in an essay

    Args:
      Essay: Essay of each student 
    
    Returns: 
      int  
  """ 
  word_no = nltk.word_tokenize(Essay)
  return len(word_no)

In [12]:
Data_Essay_01['Word_Count'] = Data_Essay_01['Essay'].apply(Word_Count)
Data_Essay_01.sample()

,essay_id,essay_set,Essay,domain1_predictionid,predicted_score,Sent_Count,Word_Count
327,2119,1,"Dear Newspaper People, Computers are one if no...",2119,9,19,318


#### Counting Characters per Essay

In [13]:
def Char_Count(Essay):
  """
    Counts characters in an essay

    Args:
      Essay: Essay of each student 
    
    Returns: 
      int
  """
  return len([character for character in Essay])

In [14]:
Data_Essay_01['Char_Count'] = Data_Essay_01['Essay'].apply(Char_Count)
Data_Essay_01.sample()

,essay_id,essay_set,Essay,domain1_predictionid,predicted_score,Sent_Count,Word_Count,Char_Count
21,1809,1,"Dear @CAPS1, I believe that computers have an ...",1809,9,29,479,2139


#### Average Word Length of Essay

In [15]:
def Avg_Word_Count(Essay):
  """
    Calculates Average Word Count In An Essay Set

    Args:
      Essay: Essay of each student 
    
    Returns: 
      float
      
  """
  word_list = nltk.word_tokenize(Essay)
  total = sum(map(len, word_list))/len(word_list)
  return total

In [16]:
Data_Essay_01['Avg_Word_Count'] = Data_Essay_01['Essay'].apply(Avg_Word_Count)
Data_Essay_01.sample()

,essay_id,essay_set,Essay,domain1_predictionid,predicted_score,Sent_Count,Word_Count,Char_Count,Avg_Word_Count
131,1920,1,"Dear Newspaper, @CAPS1 opinion on the effect(s...",1920,9,20,446,2265,4.170404


#### Average Sentence Length of Essay

In [17]:
def Average_Sentence_Length(Word_Count,Sent_Count):
  
  average_sentence_length = float(Word_Count / Sent_Count)
  return average_sentence_length

In [18]:
Data_Essay_01['Avg_Sentence_Length']=Data_Essay_01.apply(lambda row :Average_Sentence_Length(row["Word_Count"],row["Sent_Count"]),axis=1)

In [19]:
Data_Essay_01 


,essay_id,essay_set,Essay,domain1_predictionid,predicted_score,Sent_Count,Word_Count,Char_Count,Avg_Word_Count,Avg_Sentence_Length
0,1788,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788,7,13,275,1365,4.138182,21.153846
1,1789,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789,8,20,353,1605,3.671388,17.650000
2,1790,1,"Dear Local newspaper, Have you been spending a...",1790,9,15,471,2343,4.040340,31.400000
3,1791,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791,9,24,410,1981,3.958537,17.083333
4,1792,1,"Dear newspaper, I strongly believe that comput...",1792,9,34,496,2517,4.163306,14.588235
...,...,...,...,...,...,...,...,...,...,...
584,2377,1,"Dear Newspaper, @CAPS1 of people are starting ...",2377,10,31,557,2839,4.195691,17.967742
585,2378,1,"Dear @CAPS1, In this @DATE1, computors are bec...",2378,8,19,388,1993,4.244845,20.421053
586,2379,1,Well I think that using computers help some pe...,2379,5,5,128,609,3.804688,25.600000
587,2380,1,"Dear @ORGANIZATION1, Some experts and some eve...",2380,8,22,311,1532,4.051447,14.136364


#### Counting Syllables
       Primarily used in Readability scores.

In [20]:
def Syllable_Count(text):
  return textstat.syllable_count(text, lang='en_US')

In [21]:
Data_Essay_01['Preprocessed_Essay'] = Data_Essay_01['Essay'].apply(Remove_NER)
Data_Essay_01['Preprocessed_Essay'] = Data_Essay_01['Preprocessed_Essay'].apply(Tokenize_Essay)
Data_Essay_01["Syllable_Count"]=Data_Essay_01['Preprocessed_Essay'].apply(Syllable_Count)


#### Percentage of Difficult Words
    Primarily used in Readability scores.

In [22]:
def Difficult_Words_Count(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)

    # Find all words in the text
    words = []
    sentences = nltk.sent_tokenize(text)
    for sentence in sentences:
        tokens=nltk.word_tokenize(sentence)
        for token in tokens:
          words.append(token)

    # difficult words are those with syllables >= 2 ,Easy words are provided by textstat library.
    diff_words_set = set()

    #Syllabes are those words which have a break in between for example , book has a single syllable while reading has two.
    for word in words:
      syllable_count=Syllable_Count(word)
      if word not in nlp.Defaults.stop_words and syllable_count >= 2:
            diff_words_set.add(word)
 
    return len(diff_words_set)





In [23]:
Data_Essay_01["Difficult_Word_Count"]=Data_Essay_01["Preprocessed_Essay"].apply(Difficult_Words_Count)

## **Parts Of Speech Counts**

This section will cover:


*   Counting Nouns per Essay
*   Counting Adjectives per Essay
*   Counting Proper Nouns per Essay
*   Counting Adverbs per Essay
*   Counting Conjunctions per Essay

Removing NERs, Punctuations and Lower Casing

In [24]:
Data_Essay_01['Preprocessed_Essay'] = Data_Essay_01['Essay'].apply(Remove_NER)
Data_Essay_01['Preprocessed_Essay'] = Data_Essay_01['Preprocessed_Essay'].apply(Tokenize_Essay)
Data_Essay_01.head()

,essay_id,essay_set,Essay,domain1_predictionid,predicted_score,Sent_Count,Word_Count,Char_Count,Avg_Word_Count,Avg_Sentence_Length,Preprocessed_Essay,Syllable_Count,Difficult_Word_Count
0,1788,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788,7,13,275,1365,4.138182,21.153846,Dear more and more people start to use compute...,328,35
1,1789,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789,8,20,353,1605,3.671388,17.650000,Dear Time me tell you what I think of computer...,410,27
2,1790,1,"Dear Local newspaper, Have you been spending a...",1790,9,15,471,2343,4.040340,31.400000,Dear Local newspaper Have you been spending a ...,589,49
3,1791,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791,9,24,410,1981,3.958537,17.083333,Dear Readers you imagine how life would be if ...,469,44
4,1792,1,"Dear newspaper, I strongly believe that comput...",1792,9,34,496,2517,4.163306,14.588235,Dear newspaper I strongly believe that compute...,634,58


In [25]:
def Pos_Tag_Count(Essay):
  """
    Counts Parts of Speech in an Essay

    Args:
      Essay: Essay of each student 
    
    Returns: 
      int,int,int,int,int,int    
  """
  tagged_doc = nlp(Essay)

  adj_count=0
  verb_count=0
  noun_count=0
  pNoun_count=0
  adverb_count=0
  conj_count=0

  for token in tagged_doc:

    if(token.pos_ == 'ADJ'):
      adj_count+=1
    
    elif(token.pos_ =='NOUN'):
      noun_count+=1

    elif (token.pos_ =='PRON'):
      pNoun_count+=1

    elif (token.pos_ =='VERB'):
      verb_count+=1

    elif (token.pos_ =='ADV'):
      adverb_count+=1
    
    elif(token.pos_=='CCONJ'):
      conj_count+=1

  return verb_count,noun_count, adj_count, conj_count, adverb_count,pNoun_count

In [26]:
Data_Essay_01['Verb_Count'], Data_Essay_01['Noun_Count'], Data_Essay_01['Adj_Count'], Data_Essay_01['Conj_Count'], Data_Essay_01['Adverb_Count'], Data_Essay_01['pNoun_Count']=zip(*Data_Essay_01["Preprocessed_Essay"].map(Pos_Tag_Count))
Data_Essay_01.sample()

,essay_id,essay_set,Essay,domain1_predictionid,predicted_score,Sent_Count,Word_Count,Char_Count,Avg_Word_Count,Avg_Sentence_Length,Preprocessed_Essay,Syllable_Count,Difficult_Word_Count,Verb_Count,Noun_Count,Adj_Count,Conj_Count,Adverb_Count,pNoun_Count
107,1896,1,"Dear local Newspaper, I think computers have g...",1896,8,14,277,1441,4.288809,19.785714,Dear local Newspaper I think computers have gr...,344,30,40,68,19,14,14,26


In [27]:
Data_Essay_01.to_csv("01_Features.csv")

# **Evaluating Writing Attributes**

This section will cover:


*   Style
*   Content
*   Semantic
*   Semantic Coherence & Consistency 
*   Connectivity
*   Readibility Scores


## **Style**

This section will cover:


*   Mechanics
*   Grammar
*   Lexical Sophistication



### **Mechanics**

This section will cover:


*   Counting Spelling Mistakes
*   Correcting Spelling Mistakes
*   Checking Punctuations
*   Counting Punctuations
*   Checking Capitalization



#### Counting Spelling Mistakes

In [ ]:
def Check_Spelling(Sentence):
  """
    Checks spelling of each word

    Args:
      word: Words (Tokens) of each essay 
    
    Returns: 
      int
  """
  count = 0
  Sentence = word_tokenize(Sentence)
  for word in Sentence:
    word = Word(word)
  
    result = word.spellcheck()

    # result [0][0] contains the bool value if the spelling is correct or not
    # result [0][1] contains the confidence for the suggest correct spelling

    if word != result[0][0]:
      if(result[0][1] > 0.9 and not(wordnet.synsets(word)) and not("/" in word) and not (word == "If" or word == "if")):
        count = count + 1

  return count

In [ ]:
Data_Essay_01["Preprocessed_Essay"] = Data_Essay_01["Essay"].apply(Remove_NER)

In [ ]:
Data_Essay_01["Spelling_Mistakes_Count"]  = Data_Essay_01["Preprocessed_Essay"].map(Check_Spelling)
Data_Essay_01.sample()

#### Correcting Spelling Mistakes via LanguageTool
     This is for Grammer

In [ ]:
def Spelling_Error_Correct(essays):
    matches = tool.check(essays)
    is_bad_rule = lambda rule: rule.category == 'GRAMMAR'
    matches = [rule for rule in matches if not is_bad_rule(rule)]
    # print(matches[0].category)
    language_tool_python.utils.correct(essays, matches)   # to correct it
    return essays

In [ ]:
Data_Essay_01["Without_White_Spaces"] = Data_Essay_01["Essay"].apply(Remove_White_Spaces)
Data_Essay_01['Essay_SpellingCorrected_LT'] = Data_Essay_01['Without_White_Spaces'].apply(Spelling_Error_Correct)

#### Counting Number Of Punctuations

In [28]:
def Count_Punctuations(Essay):
  """
    Counts Punctuations used in an Essay

    Args:
      Essay: Essay of each student 
    
    Returns: 
      int,int,int,int,int
      
  """
  count_fullstops = 0
  count_exclamation = 0
  count_comma = 0
  count_hyphens = 0
  count_questionmark = 0

  tokens = word_tokenize(Essay)

  for word in tokens:
    if word == ".":
      count_fullstops += 1
    elif word == "!":
      count_exclamation += 1
    elif word == "?":
      count_questionmark += 1
    elif word == ",":
      count_comma += 1
    elif word == "-":
      count_hyphens += 1

  return count_fullstops , count_exclamation , count_comma , count_questionmark , count_hyphens

In [29]:
Data_Essay_01["Count_Fullstops"] , Data_Essay_01["Count_Exclamation"] , Data_Essay_01["Count_Comma"] , Data_Essay_01["Count_Questionmark"] , Data_Essay_01["Count_Hyphens"] = zip(*Data_Essay_01["Essay"].map(Count_Punctuations))
Data_Essay_01.sample()

,essay_id,essay_set,Essay,domain1_predictionid,predicted_score,Sent_Count,Word_Count,Char_Count,Avg_Word_Count,Avg_Sentence_Length,...,Noun_Count,Adj_Count,Conj_Count,Adverb_Count,pNoun_Count,Count_Fullstops,Count_Exclamation,Count_Comma,Count_Questionmark,Count_Hyphens
248,2039,1,"Dear Local Newspaper, @CAPS1 on more and more ...",2039,8,30,393,1889,3.954198,13.1,...,74,46,24,15,26,29,0,18,1,0


#### Checking Capitalization Mistakes

In [37]:
def Check_Capitalization(Essay):
  """
    Checks capitalization in each sentence of an essay

    Args:
    Essay: Words (Tokens) of each essay 

    Returns: 
    int

  """
  count = 0

  words = Essay.split()
  alreadyCounted_Words = []
  
  for i in range(len(words) - 1):
    if (i == 0):                                                    # Checking Capital Letter at the start of Sentence
      if words[i] != words[i].title():
        alreadyCounted_Words.append(words[i])
        count = count + 1
    elif "@" in words[i]:
      continue
    elif words[i] == '.' or words[i] == '"':                         # Checking Capital Letters in start of every sentence & start of every quote
      match = words[i+1]
      if match != words[i+1].title():
        alreadyCounted_Words.append(words[i])
        count = count + 1
        i = i + 1
    
    # Check if capital in middle 
    
  # Checking if all proper nouns are capital or not

  tagged_sent = nlp(Essay)

  for i in range(len(tagged_sent)):
    if tagged_sent[i].pos_ == "PROPN":
      # print(tagged_sent[i]) 
      word = tagged_sent[i].text 
      if word in alreadyCounted_Words:
        alreadyCounted_Words.remove(word)
      elif word != word.title():
        count = count + 1

  return count

In [38]:
Data_Essay_01["Preprocessed_Essay"] =  Data_Essay_01["Essay"].apply(Remove_NER)
Data_Essay_01["Capitalization_Errors"] = Data_Essay_01["Preprocessed_Essay"].apply(Check_Capitalization)
Data_Essay_01.sample()

,essay_id,essay_set,Essay,domain1_predictionid,predicted_score,Sent_Count,Word_Count,Char_Count,Avg_Word_Count,Avg_Sentence_Length,...,Adj_Count,Conj_Count,Adverb_Count,pNoun_Count,Count_Fullstops,Count_Exclamation,Count_Comma,Count_Questionmark,Count_Hyphens,Capitalization_Errors
286,2077,1,Dear Local newspaper. I'm writing this letter ...,2077,8,27,340,1656,3.991176,12.592593,...,12,9,6,54,27,0,2,0,0,1


In [39]:
# Data_Essay_01.to_csv("EssaySet01_Features_Updated.csv")     

#### Checking Punctuation Mistakes **(Incomplete)**

Checking Punctuation Mistakes

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification , pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('oliverguhr/fullstop-punctuation-multilang-large')
model = AutoModelForTokenClassification.from_pretrained('oliverguhr/fullstop-punctuation-multilang-large')
pun = pipeline('ner' , model = model , tokenizer = tokenizer)

In [ ]:
tags = pun(text)

Updated_string = ''

for output in tags:
  result = output['word'].replace('▁' , ' ') + output['entity'].replace('0', '')
  Updated_string += result

Updated_string

In [ ]:
def Punctuation_Errors(essays):
    matches = tool.check(essays)
    is_bad_rule = lambda rule: rule.category == 'PUNCTUATION'
    matches = [rule for rule in matches if is_bad_rule(rule)]
    return len(matches)

In [ ]:
Data_Essay_01["Without_White_Spaces"] = Data_Essay_01["Essay"].apply(Remove_White_Spaces)

In [ ]:
Data_Essay_01["Punctuation_Errors"] = Data_Essay_01["Without_White_Spaces"].apply(Punctuation_Errors)

### **Grammar Error Detection**

In [ ]:
# from nltk.translate.bleu_score import sentence_bleu
# reference = result.text.split()

# candidate = 'Dear local newspaper, @CAPS1 best friend, @LOCATION2, was once a nerd with no hand-eye coordination, @CAPS2, he started to use a computer and now he has better hand-eye coordination than me.'.split()
# print('BLEU score -> {}'.format(sentence_bleu(reference, candidate )))

In [ ]:
df1 = Data_Essay_01[['Essay_SpellingCorrected_LT', 'Sent_Count']]
# df1['Essay_Spelling_Corrected_LT'] = df1['Essay_SpellingCorrected_LT'].apply(Remove_White_Spaces)   # to avoid whitespace error
df1.head()

In [ ]:
def Grammar_Errors(essays):
    
    matches = tool.check(essays)
    is_bad_rule = lambda rule: rule.category == 'GRAMMAR'
    matches = [rule for rule in matches if is_bad_rule(rule)]
    # print(matches[0].category)
    errors = []
    #language_tool_python.utils.correct(text, matches)   # to correct it
    for i in range(0, len(matches)):
      errors.append(matches[i].ruleId)  # or category of the error (Misc, Whitespace, Typography)
    return len(matches), errors

In [ ]:
Grammar_Errors("Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the economy, sea floor spreading or even about the @DATE1's you'll be surprise at how much he/she knows. Believe it or not the computer is much interesting then in class all day reading out of books. If your child is home on your computer or at a local library, it's better than being out with friends being fresh, or being perpressured to doing something they know isnt right. You might not know where your child is, @CAPS2 forbidde in a hospital bed because of a drive-by. Rather than your child on the computer learning, chatting or just playing games, safe and sound in your home or community place. Now I hope you have reached a point to understand and agree with me, because computers can have great effects on you or child because it gives us time to chat with friends/new people, helps us learn about the globe and believe or not keeps us out of troble. Thank you for listening.")

In [ ]:
# Data_Essay_01['Grammar_Errors'], Data_Essay_01['Grammar_Error_List'] = zip(*df1_copy['Essay'].map(grammar_errors))
Data_Essay_01['Grammar_Error_Count'], Data_Essay_01['Grammar_Error_List'] = zip(*df1['Essay_SpellingCorrected_LT'].map(Grammar_Errors))

In [ ]:
Data_Essay_01.columns

In [ ]:
out = Data_Essay_01['Grammar_Error_List'].explode().value_counts()
out

In [ ]:
out.to_csv('GrammarErrors.csv')

In [ ]:
features = Data_Essay_01[['Sent_Count', 'Word_Count', 'Char_Count', 'Avg_Word_Count','Verb_Count', 'Noun_Count', 'Adj_Count',
       'Conj_Count', 'Adverb_Count', 'pNoun_Count', 'Grammar_Error_Count', 'Grammar_Error_List']]
features.to_csv("EssaySet01_Features.csv")     

### **Lexical Sophistication NOT DONE**

In [ ]:
pip install taaled
#RESOURCES FOR LEXICAL SOPHISTICATION 
#https://eli-data-mining-group.github.io/Pitt-ELI-Corpus/publications/Naismith_2019.pdf
#https://pypi.org/project/taaled/
#https://github.com/LCR-ADS-Lab/pylats

## **Content**

This section will cover:


*   Latent Semantic Analysis (LSA)


### **Latent Semantic Analysis (LSA)**

Content analysis generally implies only a high-level semantic analysis and comparison with source text and graded essays

## **Domain Information Content**
It is perhaps the most marked feature of our model as it tries to understand semantics and information content of an essay. To get this feature working, we first figured out the best essay from each set (highest scored essay) then, we pulled out nouns from that essay. These nouns were served as keywords for the particular domain. Then, we fire these word into 'WordNet' and take out their equivalent. In this way, for each set, we got a bunch of different words, most relevant to its particular domain. Then, we count the number of domain words in the essay provided. 

In [ ]:
temp_df = Data_Essay_01[Data_Essay_01['Total Score'] == max(Data_Essay_01["Total Score"])]
temp_df.head()

In [ ]:
temp_df['Preprocessed_Essay'] = temp_df['Essay'].apply(Remove_NER)
temp_df['Preprocessed_Essay'] = temp_df['Preprocessed_Essay'].apply(Tokenize_Essay)
temp_df.head()

In [ ]:
Nouns_List = []

# Extracting Nouns of Best Essays

def Extract_Nouns(Essay):
  """
    Extract Nouns From an Essay

    Args:
      Essay: Best Essay based on score  
  """
  tagged_doc = nlp(Essay)

  for token in tagged_doc:
    if(token.pos_ =='NOUN'):
      if (not(token in Nouns_List)):
        Nouns_List.append(token)

In [ ]:
temp_df["Preprocessed_Essay"].apply(Extract_Nouns)
Nouns_List

## **Semantic Similarity**
https://towardsdatascience.com/how-to-compute-sentence-similarity-using-bert-and-word2vec-ab0663a5d64

In [ ]:
!pip install sent2vec

In [ ]:
from scipy import spatial
from sent2vec.vectorizer import Vectorizer

In [ ]:
#@title Default title text
# Extracting Sentences  
vectorizer = Vectorizer(pretrained_weights = 'distilbert-base-multilingual-cased')

def Extract_Sentences(Essay):
  sentences = sent_tokenize(Essay)
  return sentences

def Apply_Embedding(Sentences):
  vectorizer.run(Sentences, remove_stop_words=['not'], add_stop_words=[])
  vectors_w2v = vectorizer.vectors
  distances = []
  for i in range(len(vectors_w2v) - 1):
    distances.append(spatial.distance.cosine(vectors_w2v[i], vectors_w2v[i+1]))
  Average = sum(distances) / len(distances)

  return Average

In [ ]:
Data_Essay_01["Sentences_Tokenized"] = Data_Essay_01['Essay'].apply(Extract_Sentences)

In [ ]:
Data_Essay_01["Semantic_Similarity"]  = Data_Essay_01["Sentences_Tokenized"].map(Apply_Embedding)

In [ ]:
Data_Essay_01.sample

In [ ]:
Data_Essay_01.to_csv('output.csv')

## **Connectivity**

In [ ]:
# Step01: Making a temp dataframe for sepearte preprocessing of connectors
df_connectors = Data_Essay_01[['Essay', 'Sent_Count']]
df_connectors['Preprocessed_Essay_For_Connectors'] = df_connectors['Essay'].apply(LowerCase_Words)  # for ease in matching
df_connectors['Preprocessed_Essay_For_Connectors'] = df_connectors['Preprocessed_Essay_For_Connectors'].apply(Remove_NER) # no impact on connectors count, removing it to save computational time.
df_connectors['Preprocessed_Essay_For_Connectors'] = df_connectors['Preprocessed_Essay_For_Connectors'].apply(Remove_White_Spaces) # for ease in ngrams
df_connectors.head()


In [ ]:
#Step02: Generating n_grams of each essay. 
def generate_N_grams(essay,ngram=1):
  words=[word for word in essay.split(" ")]  
  temp=zip(*[words[i:] for i in range(0,ngram)])
  ngram_list=[' '.join(ngram) for ngram in temp]
  return ngram_list

In [ ]:
df_connectors['Unigrams'] = df_connectors['Preprocessed_Essay_For_Connectors'].apply(generate_N_grams, ngram =1)
df_connectors['Bigrams'] = df_connectors['Preprocessed_Essay_For_Connectors'].apply(generate_N_grams, ngram =2)
df_connectors['Trigrams'] = df_connectors['Preprocessed_Essay_For_Connectors'].apply(generate_N_grams, ngram =3)
df_connectors['Four-grams'] = df_connectors['Preprocessed_Essay_For_Connectors'].apply(generate_N_grams, ngram =4)
df_connectors['Five-grams'] = df_connectors['Preprocessed_Essay_For_Connectors'].apply(generate_N_grams, ngram =5)

In [ ]:
df_connectors['Four-grams'][17]

In [ ]:
# Step03: Loading the file of connectors.
filename = "/content/drive/MyDrive/IntelliTech-DataSet/Connectors_Words.txt"
with open(filename) as file:
    defined_connectors = [line.rstrip() for line in file]

defined_connectors = set(defined_connectors)  #remove duplicate connectors
print(len(defined_connectors))

In [ ]:
# Step 04: Make a connector detection function

def connectors_detection(unigrams_list, bigram_list, trigram_list, four_grams_list, five_grams_list):
  connectors_detected = []
  for token in unigrams_list:
    if token in defined_connectors:
      connectors_detected.append(token)
  for token in bigram_list:
    if token in defined_connectors:
      connectors_detected.append(token)
  for token in trigram_list:
    if token in defined_connectors:
      connectors_detected.append(token)
  for token in four_grams_list:
    if token in defined_connectors:
      connectors_detected.append(token)
  for token in five_grams_list:
    if token in defined_connectors:
      connectors_detected.append(token)
  
  #use SET connectors_detected for tf-idf work, unique connectors
  return set(connectors_detected)
  #return len(connectors_detected)

In [ ]:
df_connectors['Connectors'] = df_connectors.apply(lambda row: connectors_detection(row['Unigrams'], row['Bigrams'],row['Trigrams'],row['Four-grams'],row['Five-grams']),axis = 1)

In [ ]:
df_connectors.head()

The minimum value that the index may generate is 0. There is no specified maximum limit on the index. Zero (0) denotes no connector at all, while an index close to 2 shows that there are many effective connectors in (almost) every sentence. The connectivity index is calculated for each essay.

In [ ]:
# Optimizing connectors via TF-IDF

def TF(essay,connectors):
  freq = 0
  for word in essay.split(" "):
    if word in connectors:
      freq = freq + 1
  return freq

def IDF(essay,connector):
  doc = 0;
  sent_list = nltk.tokenize.sent_tokenize(essay)
  for i in range(0, len(sent_list)):
    if connector in sent_list[i]:
      doc = doc + 1
  return 1+ np.log(len(sent_list) / (doc))

def TF_IDF(essay, connector):
  tf = TF(essay,connector)
  idf = IDF(essay,connector)
  tf_idf = tf * idf
  return tf_idf

def connectivity_index(essay, connectors,num_sent):
  weights = []
  for word in connectors:
    score = TF_IDF(essay,word)
    weights.append(score)
  return float(sum(weights) / num_sent)

Data_Essay_01['Connectivity_Index'] = df_connectors.apply(lambda row: connectivity_index(row['Preprocessed_Essay_For_Connectors'], row['Connectors'], row['Sent_Count']), axis=1)


In [ ]:
df_connectors.head()

In [ ]:
Data_Essay_01['Connectivity_Index'].describe()

In [ ]:
Data_Essay_01['Connectivity_Index'].hist()

## **Readibility Scores**

We section will cover:
1.   Flesch Reading Ease
2.   Flesch-Kincaid Grade Level
3.   Gunning Fog Index
4.   Dale Chall Readability Formula

In [30]:
#Required Pre-processing for Readability Scores.
Data_Essay_01["Preprocessed_Essay"] = Data_Essay_01["Essay"].apply(Remove_NER2)
Data_Essay_01["Preprocessed_Essay"] = Data_Essay_01["Preprocessed_Essay"].apply(LowerCase_Words)

In [31]:
Data_Essay_01

,essay_id,essay_set,Essay,domain1_predictionid,predicted_score,Sent_Count,Word_Count,Char_Count,Avg_Word_Count,Avg_Sentence_Length,...,Noun_Count,Adj_Count,Conj_Count,Adverb_Count,pNoun_Count,Count_Fullstops,Count_Exclamation,Count_Comma,Count_Questionmark,Count_Hyphens
0,1788,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788,7,13,275,1365,4.138182,21.153846,...,57,22,11,20,12,12,0,21,1,0
1,1789,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789,8,20,353,1605,3.671388,17.650000,...,57,12,14,29,53,19,1,1,0,0
2,1790,1,"Dear Local newspaper, Have you been spending a...",1790,9,15,471,2343,4.040340,31.400000,...,76,19,16,30,79,13,0,6,2,0
3,1791,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791,9,24,410,1981,3.958537,17.083333,...,74,11,15,19,47,17,2,8,5,0
4,1792,1,"Dear newspaper, I strongly believe that comput...",1792,9,34,496,2517,4.163306,14.588235,...,113,38,18,31,57,34,0,9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,2377,1,"Dear Newspaper, @CAPS1 of people are starting ...",2377,10,31,557,2839,4.195691,17.967742,...,107,38,29,24,59,31,0,13,0,0
585,2378,1,"Dear @CAPS1, In this @DATE1, computors are bec...",2378,8,19,388,1993,4.244845,20.421053,...,94,20,9,21,36,18,0,17,1,0
586,2379,1,Well I think that using computers help some pe...,2379,5,5,128,609,3.804688,25.600000,...,23,10,11,6,17,5,0,0,0,0
587,2380,1,"Dear @ORGANIZATION1, Some experts and some eve...",2380,8,22,311,1532,4.051447,14.136364,...,61,20,10,15,28,18,4,3,0,0


### Flesch Reading Ease Score
    In the Flesch reading-ease test, higher scores indicate material that is easier to read; lower numbers mark passages that are more difficult to read

In [32]:
def Flesch_Reading_Ease_Score(syllable_Count,NoOfsentences,total_Words):
  return (206.835-1.015*(total_Words/float(NoOfsentences))-84.6*(syllable_Count / float(total_Words)))

In [33]:
Data_Essay_01['Flesch_Reading_Ease_Score']=Data_Essay_01.apply(lambda row :Flesch_Reading_Ease_Score(row["Syllable_Count"],row["Sent_Count"],row["Word_Count"]),axis=1)

### Flesch-Kincaid Grade
    Used extensively in educational fields.
  
    It can also mean the number of years of education generally required to understand this text, relevant when the formula results in a number greater than 10.
    
    For example, a score of 9.3 means that a ninth grader would be able to read the document.






In [34]:
def Flesh_Kincaid_Grade(total_Words,total_Sentences,syllable_Count):
   F = float(0.39 * (total_Words / total_Sentences) + 11.8 * (syllable_Count / float(total_Words)) - 15.59)
   return F

In [35]:
Data_Essay_01['Flesh_Kincaid_Grade']=Data_Essay_01.apply(lambda row :Flesh_Kincaid_Grade(row["Syllable_Count"],row["Sent_Count"],row["Word_Count"]),axis=1)

###[Dale-Chall Readability Score](https://www.kaggle.com/code/fshenbayh/readability)
    Provides a numeric gauge of the comprehension difficulty that readers come upon when reading a text
    For example , if score is 4.9 or lower then text is understood by average 4th grade student or lower.







In [36]:
#Raw score = 0.1579*(PDW) + 0.0496*(ASL) if the percentage of PDW is less than 5%, otherwise compute
#Raw score = 0.1579*(PDW) + 0.0496*(ASL) + 3.6365  

def Dale_Challl_Readability_Score(word_Count ,difficult_Words,avg_sent_length):
    NOT_DIFFICULT_WORDS = word_Count - difficult_Words
    if(word_Count >0):
      # Percentage of words not on difficult word list

      percent_Not_Difficult_Words = float(NOT_DIFFICULT_WORDS) / float(word_Count) * 100
        # diff_words stores percentage of difficult words
    diff_words = 100 - percent_Not_Difficult_Words
    Dale_Chall_Score = (0.1579 * diff_words) +  (0.0496 * avg_sent_length)
    if diff_words > 5:      
 
        Dale_Chall_Score += 3.6365
         
    return Dale_Chall_Score




In [37]:
Data_Essay_01['Dale_Challl_Readability_Score']=Data_Essay_01.apply(lambda row :Dale_Challl_Readability_Score(row["Word_Count"],row["Difficult_Word_Count"],row["Avg_Sentence_Length"]),axis=1)

In [38]:
Data_Essay_01 

,essay_id,essay_set,Essay,domain1_predictionid,predicted_score,Sent_Count,Word_Count,Char_Count,Avg_Word_Count,Avg_Sentence_Length,...,Adverb_Count,pNoun_Count,Count_Fullstops,Count_Exclamation,Count_Comma,Count_Questionmark,Count_Hyphens,Flesch_Reading_Ease_Score,Flesh_Kincaid_Grade,Dale_Challl_Readability_Score
0,1788,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788,7,13,275,1365,4.138182,21.153846,...,20,12,12,0,21,1,0,84.459119,4.143293,6.695367
1,1789,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789,8,20,353,1605,3.671388,17.650000,...,29,53,19,1,1,0,0,90.659627,2.564512,5.719674
2,1790,1,"Dear Local newspaper, Have you been spending a...",1790,9,15,471,2343,4.040340,31.400000,...,30,79,13,0,6,2,0,69.169096,9.159993,6.836636
3,1791,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791,9,24,410,1981,3.958537,17.083333,...,19,47,17,2,8,5,0,92.721270,2.346815,6.178370
4,1792,1,"Dear newspaper, I strongly believe that comput...",1792,9,34,496,2517,4.163306,14.588235,...,31,57,34,0,9,0,0,83.890038,0.913899,6.206488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,2377,1,"Dear Newspaper, @CAPS1 of people are starting ...",2377,10,31,557,2839,4.195691,17.967742,...,24,59,31,0,13,0,0,79.392356,2.596791,6.823912
585,2378,1,"Dear @CAPS1, In this @DATE1, computors are bec...",2378,8,19,388,1993,4.244845,20.421053,...,21,36,18,0,17,1,0,80.793714,3.803300,7.294616
586,2379,1,Well I think that using computers help some pe...,2379,5,5,128,609,3.804688,25.600000,...,6,17,5,0,0,0,0,73.779125,6.369457,6.509932
587,2380,1,"Dear @ORGANIZATION1, Some experts and some eve...",2380,8,22,311,1532,4.051447,14.136364,...,15,28,18,4,3,0,0,91.020996,0.860879,6.470075


# **Features Correlation Visualization**

In [ ]:
Data_Essay_01_Features = pd.read_csv("/content/drive/MyDrive/IntelliTech-DataSet/EssaySet01_Features.csv")
Data_Essay_01_Features.head()

In [ ]:
Data_Essay_01_Features["Target_Score"] = Data_Essay_01["Total Score"]
Data_Essay_01_Features["Flesch_Reading_Score"] = Data_Essay_01["Flesch_Reading_Score"]
Data_Essay_01_Features.columns

In [ ]:
Data_Essay_01_Features.drop('Unnamed: 0' , axis = 1 , inplace = True)
Data_Essay_01_Features.drop("Unnamed: 20" , axis = 1, inplace = True)
Data_Essay_01_Features.drop("Unnamed: 18" , axis = 1, inplace = True)

In [ ]:
Data_Essay_01_Features.corr()

In [ ]:
import seaborn as sns

# Plot correlation of essay-length related features
cols = ['Sent_Count', 'Word_Count', 'Char_Count']
#        'Avg_Word_Count',
#        'Verb_Count', 'Noun_Count', 'Adj_Count', 'Conj_Count', 'Adverb_Count',
#        'pNoun_Count', 'Count_Fullstops', 'Count_Exclamation', 'Count_Comma',
#        'Count_Questionmark', 'Count_Hyphens', 'Capitalization_Errors',
#        'Grammar_Error_Count', 'Punctuation_Errors']
        
g = sns.pairplot(Data_Essay_01_Features, hue='Target_Score', vars=cols, plot_kws={"s": 20}, palette="bright")
g.fig.subplots_adjust(top=.93)
g.fig.suptitle('Pairplots of select features', fontsize=16)
plt.show()

#**Vocabulary Analysis**

In [39]:
#REPEATED PROCESSING AS BAWANY SAID EVERYONE GOT TO DO THEIR OWN PROCESSING.
Data_Essay_01["Preprocessed_Essay"] = Data_Essay_01["Essay"].apply(Remove_NER2)
Data_Essay_01["Preprocessed_Essay"] = Data_Essay_01["Preprocessed_Essay"].apply(LowerCase_Words)
Data_Essay_01["Preprocessed_Essay"] = Data_Essay_01["Preprocessed_Essay"].apply(Remove_Punctuations)



#### Lexical Diversity ( Ratio of Unique Words to Total Number of Words ) 

In [40]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

def Unique_Words_Per_Essay(text):

  token_words = word_tokenize(text)

    #Perform lemma on token_words( since Continous and Continuously are both one uniquen word) and then join them as a sentence.

  unique_words_lemma=[]
  for unique in token_words:
    unique_words_lemma.append(lemmatizer.lemmatize(unique, 'v'))
    unique_words_lemma.append(" ")
  lemma_words= "".join(unique_words_lemma)

  #COUNT ALL UNIQUE WORDS IN LEMMATIZED WORDS
  unique_words = set(lemma_words.split())

  #Neglect all the misspelt words
  correct_unique_words=[]

  for unique in unique_words:
    word = Word(unique)
    result = word.spellcheck()

  # #   # result [0][0] contains the bool value if the spelling is correct or not
  # #   # result [0][1] contains the confidence for the suggest correct spelling

    if word == result[0][0]:
        correct_unique_words.append(word)
  length = len(correct_unique_words)

  return length

def Ratio_Lexical_Diversity(Length_Unique_Words,Word_Count):
    return Length_Unique_Words/Word_Count


In [41]:
Data_Essay_01["Unique_Words_LD"]=Data_Essay_01["Preprocessed_Essay"].apply(Unique_Words_Per_Essay) #Simple Count
Data_Essay_01["Lexical_Diversity"]=Data_Essay_01.apply(lambda row :Ratio_Lexical_Diversity(row["Unique_Words_LD"],row["Word_Count"]),axis=1) #Ratio


In [42]:
Data_Essay_01

,essay_id,essay_set,Essay,domain1_predictionid,predicted_score,Sent_Count,Word_Count,Char_Count,Avg_Word_Count,Avg_Sentence_Length,...,Count_Fullstops,Count_Exclamation,Count_Comma,Count_Questionmark,Count_Hyphens,Flesch_Reading_Ease_Score,Flesh_Kincaid_Grade,Dale_Challl_Readability_Score,Unique_Words_LD,Lexical_Diversity
0,1788,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788,7,13,275,1365,4.138182,21.153846,...,12,0,21,1,0,84.459119,4.143293,6.695367,103,0.374545
1,1789,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789,8,20,353,1605,3.671388,17.650000,...,19,1,1,0,0,90.659627,2.564512,5.719674,133,0.376771
2,1790,1,"Dear Local newspaper, Have you been spending a...",1790,9,15,471,2343,4.040340,31.400000,...,13,0,6,2,0,69.169096,9.159993,6.836636,161,0.341826
3,1791,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791,9,24,410,1981,3.958537,17.083333,...,17,2,8,5,0,92.721270,2.346815,6.178370,127,0.309756
4,1792,1,"Dear newspaper, I strongly believe that comput...",1792,9,34,496,2517,4.163306,14.588235,...,34,0,9,0,0,83.890038,0.913899,6.206488,161,0.324597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,2377,1,"Dear Newspaper, @CAPS1 of people are starting ...",2377,10,31,557,2839,4.195691,17.967742,...,31,0,13,0,0,79.392356,2.596791,6.823912,196,0.351885
585,2378,1,"Dear @CAPS1, In this @DATE1, computors are bec...",2378,8,19,388,1993,4.244845,20.421053,...,18,0,17,1,0,80.793714,3.803300,7.294616,176,0.453608
586,2379,1,Well I think that using computers help some pe...,2379,5,5,128,609,3.804688,25.600000,...,5,0,0,0,0,73.779125,6.369457,6.509932,63,0.492188
587,2380,1,"Dear @ORGANIZATION1, Some experts and some eve...",2380,8,22,311,1532,4.051447,14.136364,...,18,4,3,0,0,91.020996,0.860879,6.470075,111,0.356913


#### Lexical Density (Number of content words like POS tags)

In [43]:
def Lexical_Density_Per_Essay(Verb_Count,Adj_Count,Conj_Count,Noun_Count,Adv_Count,pNoun_Count):
  sum =0
  sum=sum+Verb_Count+Adj_Count+Conj_Count+Noun_Count+Adv_Count+pNoun_Count
  return sum;

In [44]:
Data_Essay_01["Lexical_Density"]=Data_Essay_01.apply(lambda row :Lexical_Density_Per_Essay(row["Verb_Count"],row["Adj_Count"],row["Conj_Count"],row["Noun_Count"],row["Adverb_Count"],row["pNoun_Count"]),axis=1)


#### Lexical Accuracy (Number of *errors*)

In [45]:
#LEAVE IT FOR LATER, WHEN WE WILL RUN THE NOTEBOOK FOR SPELLING AND GRAMMER MISTAKES THEN I WILL CALCULATE THE LEXICAL ACCURACY.

#### Lexical originality(# of words unique to a writer divided by total # of words in a corpous of group of writers(essay set))

In [46]:
def Total_Word_Count():
  return Data_Essay_01['Word_Count'].sum()   
def Lexical_Originality_Per_Essay(Unique_Words,Total_Words):
    return Unique_Words/Total_Words


In [47]:
Total=Total_Word_Count()
Data_Essay_01["Lexical_Originality"]=Data_Essay_01.apply(lambda row :Lexical_Originality_Per_Essay(row["Unique_Words_LD"],Total),axis=1)


In [48]:
Data_Essay_01.to_csv("01_Features.csv")